In [1]:
from sentence_transformers import SentenceTransformer
import requests 
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Q1. Getting the embeddings model

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)


In [3]:
user_question = "I just discovered the course. Can I still join it?"
user_question_vec = embedding_model.encode(user_question)

In [4]:
user_question_vec[0]

0.078222655

#### Prepare the documents

In [5]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
ml_zoomcamp_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(ml_zoomcamp_documents)

375

### Q2. Creating the embeddings

In [7]:
ml_zoomcamp_documents[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp',
 'id': '39fda9f0'}

In [9]:
embeddings = []

for doc in ml_zoomcamp_documents:
    qa_text = '{question} {text}'.format(question=doc['question'], text=doc['text'])
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)

In [10]:
X.shape

(375, 768)

### Q3. Search

In [11]:
user_question_vec.dot(user_question_vec)

0.9999999

In [12]:
scores = X.dot(user_question_vec)

In [13]:
max(scores)

0.6506573

#### Simple Vector Search

In [14]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_zoomcamp_documents, embeddings=X)

In [15]:
search_engine.search(user_question_vec, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [16]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [17]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [18]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [19]:
def vector_search_engine_wrapper(doc):
    question = doc['question']
    question_vec = embedding_model.encode(question)
    
    results = search_engine.search(question_vec, num_results=5)
    return results

In [ ]:
evaluate(ground_truth=ground_truth, search_function=vector_search_engine_wrapper)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

### Q5. Indexing with Elasticsearch

In [21]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [41]:
for i in range(len(ml_zoomcamp_documents)):
    ml_zoomcamp_documents[i]['question_text_vector'] = embeddings[i]

In [23]:
for doc in tqdm(ml_zoomcamp_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [43]:
def elastic_search_knn(field, vector, course, return_text = True):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    results = es_results

    if return_text:
        result_docs = []
        
        for hit in es_results['hits']['hits']:
            result_docs.append(hit['_source'])
        
        results = result_docs

    return results

In [44]:
results = elastic_search_knn(
    field = 'question_text_vector',
    vector = user_question_vec,
    course = 'machine-learning-zoomcamp',
    return_text = False
    )
results

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.82532895, 'hits': [{'_index': 'course-questions', '_id': 'icKLp5AB2wTLFK5JyJDK', '_score': 0.82532895, '_source': {'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'id': 'ee58a693'}}, {'_index': 'course-questions', '_id': 'jMKLp5AB2wTLFK5JyZAT', '_score': 0.73585373, '_source': {'question': 'I just joined. What should I do n

In [45]:
max_score = results['hits']['max_score']

id_of_high_scorer = [result['_source']['id'] for result in results['hits']['hits'] if result['_score'] == max_score]

id_of_high_scorer

['ee58a693']

### Q6. Hit-rate for Elasticsearch

In [46]:
def elastic_query_wrapper(doc):
    question = doc['question']
    course = doc['course']

    question_vector = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', question_vector, course, return_text=True)

In [48]:
evaluate(ground_truth=ground_truth, search_function=elastic_query_wrapper)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}